# Node Inputs

In [1]:
import json
from subprocess import call
import numpy as np

# Important parameters

# Total number of nodes
n = [4,8,16,32,64,128,256,512,1024]

# Number of witnesses
w = [int(min(10*np.log2(x),x)) for x in n]
v = [int(min(12.5*np.log2(x),x)) for x in n]
u = [x//2 + 1 for x in w]

# Number of Samples
s = [int(min(10*np.log2(x),x)) for x in n]
th = [(2*x)//3 + 1 for x in s]

In [2]:
# Brachas's inputs

for i in range(len(n)):
    experiment = {"protocol": "bracha"}
    experiment["parameters"] = {"n": n[i], "f": 0}
    
    # Serializing json
    json_object = json.dumps(experiment, indent=4)

    # Writing to sample.json
    with open("Experiments/brachaInput" + str(i) + ".json", "w") as outfile:
        outfile.write(json_object)

In [3]:
# Reliable Accountability

# Fixed Parameters
wr = 10
vr = 10
recoveryTm = 30000000000
idSize = 256
numberBins = 16

for i in range(len(n)):
    experiment = {"protocol": "reliable_accountability"}
    experiment["parameters"] = {"n": n[i], "f": 0,
                               "w": w[i], "v": v[i], "wr": 10, "vr": 10,
                               "u": u[i], "recovery_timeout": recoveryTm,
                               "node_id_size": idSize, "numberBins": numberBins}
    
    json_object = json.dumps(experiment, indent=4)

    with open("Experiments/witnessInput" + str(i) + ".json", "w") as outfile:
        outfile.write(json_object)

In [4]:
# Scalable Broadcast

for i in range(len(n)):
    experiment = {"protocol": "scalable"}
    experiment["parameters"] = {"n": n[i], "f": 0,
                               "g_size": s[i], "e_size": s[i], "r_size": s[i], "d_size": s[i],
                               "e_threshold": th[i], "r_threshold": th[i], "d_threshold": th[i]}
    
    json_object = json.dumps(experiment, indent=4)

    with open("Experiments/scalableInput" + str(i) + ".json", "w") as outfile:
        outfile.write(json_object)

# Simulation config files

In [5]:
# Changes bellow are mainly on the system's throughput
# Transactions / second
targetThr = [2**(4+i) for i in range(10)]

# Requests are evengly split among nodes according to the formula  

#    thr = n * thr_n, where thr_n is a node's requests per second

totalTime = 120000000000 # 120 seconds in nanoseconds
baseDelay = 1000000000 # 1 second in nanoseconds
for i in range(len(n)):
    for th in targetThr:
        # Fixed Parameters
        config = {"topology": "singleTopo.json",
                  "hostBand": 10, "hostQueue": 10000,
                  "hostDelay": "100ms", "hostLoss": .1}
        
        config["testNodes"] = {"inTestNodes": False, "outTestNodes": True,
                               "inTopoDelay": "25ms", "outTopoDelay": "25ms"}
        
        trDelay = (n[i]/th) * baseDelay
        totalTr = totalTime/trDelay
        
        config["simulationTime"] = int(totalTime/baseDelay) + 60
        config["numberTransactions"] = int(totalTr)
        config["transactionDelay"] = int(trDelay)
        
        json_object = json.dumps(config, indent=4)

        with open("Experiments/config" + str(i) + "_" + str(th) + ".json", "w") as outfile:
            outfile.write(json_object)